In [37]:
# Import Modules
import math
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from warnings import catch_warnings
from warnings import simplefilter
import scipy.stats as stats
import itertools
import pandas as pd
from sklearn.model_selection import train_test_split, KFold

In [38]:
# Data Treatment
data_path = "C:/Users/USER/Documents/workspace/BMED/DB_bench_BMED_for_LA.xlsx"
#data_path = "C:/Users/bsjun/Documents/workspace/BMED/DB_bench_BMED_for_LA.xlsx"
raw_Data= pd.read_excel(data_path,sheet_name="data_for_ML")

MVs = raw_Data[['T_operation','V_operation','E_operation','C_F_LA','C_A_LA']]
CVs = raw_Data[['SEC','J_LA']]

X = MVs.values
Y = CVs.values

In [39]:
# surrogate or approximation for the objective function
def surrogate(model, X):
    # catch any warning generated when making a prediction
    with catch_warnings():
        # ignore generated warnings
        simplefilter('ignore')
        return model.predict(X, return_std=True)

In [40]:
# Expected improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrgate score found so far
    yhat, _ = surrogate(model, X)
    best1, best2 = min([i[0] for i in yhat]), max([i[1] for i in yhat])

    # calculate mean and stdev via surrogate function
    mu, std = surrogate(model, Xsamples)

    # Calculate the expected improvement (EI)
    # Clip std to avoid division by zero
    std = np.clip(std, 1e-9, None)  # Replace None with a suitable upper bound if needed
    std2 = [i[0] for i in std]
    z = score2 - best / std2
    ei = (score2 - best) * stats.norm.cdf(z) + std2 * stats.norm.pdf(z)
    return ei

In [41]:
# optimize the acquisition function
def opt_acquisition(X, y, model):
    # grid search, generate samples
    Tsample = np.linspace(25,35,31)
    Vsample = np.linspace(10,35,31)
    Esample = np.linspace(0.25,1,31)
    Fsample = np.linspace(-0.1,5.2,31)
    Asample = np.linspace(-0.1,2.2,31)
    Xsamples = np.asarray(list(itertools.product(Tsample,Vsample,Esample,Fsample,Asample)))
    
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = np.argmax(scores)
    return Xsamples[ix]

In [42]:
# plot real observation vs surrogate function
def plot(X, y, model):
    # Split inputs
    Xe1 = [X[i][0] for i in range(len(X))]
    Xe2 = [X[i][1] for i in range(len(X))]
    Xe3 = [X[i][2] for i in range(len(X))]
    ye1 = [y[i][0] for i in range(len(y))]
    ye2 = [y[i][1] for i in range(len(y))]

    # scatter plot of imputs
    fig, axes = plt.subplots(2,3)
    axes[0,0].scatter(Xe1, ye1)
    axes[0,1].scatter(Xe2, ye1)
    axes[0,2].scatter(Xe3, ye1)
    axes[1,0].scatter(Xe1, ye2)
    axes[1,1].scatter(Xe2, ye2)
    axes[1,2].scatter(Xe3, ye2)

    # line plot of surragte function acorss domain
    Xp1 = np.append(Xe1,10 + 10*np.random.random(20))
    Xp2 = np.append(Xe2,2*np.random.random(20))
    Xp3 = np.append(Xe3,-10 + 20*np.random.random(20))
    Xpred = np.asarray(list(itertools.product(Xp1,Xp2,Xp3)))

    ypred, _ = surrogate(model, Xpred)
    yp1 = [i[0] for i in ypred]
    yp2 = [i[1] for i in ypred]

    Xpl1 = [i[0] for i in Xpred]
    Xpl2 = [i[1] for i in Xpred]
    Xpl3 = [i[2] for i in Xpred]

    axes[0,0].scatter(Xpl1, yp1, s=1)
    axes[0,1].scatter(Xpl2, yp1, s=1)
    axes[0,2].scatter(Xpl3, yp1, s=1)
    axes[1,0].scatter(Xpl1, yp2, s=1)
    axes[1,1].scatter(Xpl2, yp2, s=1)
    axes[1,2].scatter(Xpl3, yp2, s=1)
    # show the plot
    plt.show()

In [52]:
# Train Set Normalization
# split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)

T = [i[0] for i in X_train]
V = [i[1] for i in X_train]
E = [i[2] for i in X_train]
F = [i[3] for i in X_train]
A = [i[4] for i in X_train]

SEC = [i[0] for i in Y_train]
JLA = [i[1] for i in Y_train]

rX_train = list(range(len(X_train)))
rY_train = list(range(len(X_train)))

# Z-score normalization
for i in range(len(X_train)):
    iX = [(T[i]-np.average(T))/np.std(T),(V[i]-np.average(V))/np.std(V),(E[i]-np.average(E))/np.std(E),(F[i]-np.average(F))/np.std(F),(A[i]-np.average(A))/np.std(A)]
    iY = [(SEC[i]-np.average(SEC))/np.std(SEC),(JLA[i]-np.average(JLA))/np.std(JLA)]
    rX_train[i] = iX
    rY_train[i] = iY
rX_train = np.asarray(rX_train)
rY_train = np.asarray(rY_train)

model = GaussianProcessRegressor()
model.fit(rX_train,rY_train)

yhat, _ = surrogate(model, rX_train)
best1, best2 = min([i[0] for i in yhat]), max([i[1] for i in yhat])
print(best1)
print(best2)
Tsample = np.linspace(25,35,3)
Vsample = np.linspace(10,35,3)
Esample = np.linspace(0.25,1,3)
Fsample = np.linspace(-0.1,5.2,3)
Asample = np.linspace(-0.1,2.2,3)
Xsamples = np.asarray(list(itertools.product(Tsample,Vsample,Esample,Fsample,Asample)))

-0.8778715562075377
3.330318313931372


In [47]:
mu, std = surrogate(model, Xsamples)


In [51]:
std

array([[0.06732994, 0.06732994],
       [0.15208167, 0.15208167],
       [0.83110497, 0.83110497],
       [0.05398342, 0.05398342],
       [0.28429014, 0.28429014],
       [0.86711934, 0.86711934],
       [0.11384449, 0.11384449],
       [0.80553667, 0.80553667],
       [0.9940368 , 0.9940368 ],
       [0.36759157, 0.36759157],
       [0.38894681, 0.38894681],
       [0.8551645 , 0.8551645 ],
       [0.36567315, 0.36567315],
       [0.44877984, 0.44877984],
       [0.88568813, 0.88568813],
       [0.37741924, 0.37741924],
       [0.83363647, 0.83363647],
       [0.99482112, 0.99482112],
       [0.65787551, 0.65787551],
       [0.66587956, 0.66587956],
       [0.90762679, 0.90762679],
       [0.65717398, 0.65717398],
       [0.69012139, 0.69012139],
       [0.92662537, 0.92662537],
       [0.66151484, 0.66151484],
       [0.89439542, 0.89439542],
       [0.99660665, 0.99660665],
       [0.99905054, 0.99905054],
       [0.99903773, 0.99903773],
       [0.99953932, 0.99953932],
       [0.

In [45]:





# 5-Fold Cross Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


for train_index, val_index in kfold.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

    model = GaussianProcessRegressor()
    model.fit(X_train_fold,Y_train_fold)



# # define the model
# model = GaussianProcessRegressor()
# model.fit(X,y)

# x = opt_acquisition(X, y, model)
# actual = [67.06,49.637]
# est, _ = surrogate(model, [X[2]])
# print(f'>x = {x}, f()={est}, actual={actual}')
# print(np.exp(x[0]),np.exp(x[0])*5)


In [46]:
# best result
score3=np.zeros(len(y))
for i in range(len(y)):
        score3[i] = eval(y[i])
ix = np.argmin(score3)
print(X[ix], y[ix])

plt.scatter(score3)

,[16.0505649,1.19114408,-0.68813221]
,[67.19,49.637]

<>:10: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:10: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?
<>:11: SyntaxWarning: 'list' object is not callable; perhaps you missed a comma?


NameError: name 'y' is not defined